# Fuel efficiency data #

Aim: to investigate how the addition of Hydrotreated Vegetable Oil (HVO) to diesel fuel changes fuel efficiency based on 2024 data

## Importing the libraries and pre-preparing the data ##

In [2]:
import pandas as pd

In [4]:
# Hydrogenated vegetable oil tank
HVO_tank = pd.read_csv('OWN tank 2024 data.csv')

In [5]:
# Making sure it's loaded correctly
HVO_tank

,DATE,LITRES,REG,Country,Type
0,02/01/2024,67.31,truck32,GB,HVO
1,02/01/2024,45.97,truck29,GB,HVO
2,02/01/2024,52.02,truck24,GB,HVO
3,02/01/2024,236.74,truck6,GB,HVO
4,02/01/2024,329.21,truck4,GB,HVO
...,...,...,...,...,...
4494,23/12/2024,104.16,truck22,GB,HVO
4495,23/12/2024,58.34,0,GB,HVO
4496,24/12/2024,30.97,0,GB,HVO
4497,31/12/2024,266.53,0,GB,HVO


Where the value of `REG` is a zero, it is referring to another vehicle type, such as a forklift and it should be dropped.

In [6]:
HVO_tank = HVO_tank[HVO_tank['REG'] != '0'].copy()
HVO_tank

,DATE,LITRES,REG,Country,Type
0,02/01/2024,67.31,truck32,GB,HVO
1,02/01/2024,45.97,truck29,GB,HVO
2,02/01/2024,52.02,truck24,GB,HVO
3,02/01/2024,236.74,truck6,GB,HVO
4,02/01/2024,329.21,truck4,GB,HVO
...,...,...,...,...,...
4491,23/12/2024,69.20,truck30,GB,HVO
4492,23/12/2024,63.23,truck32,GB,HVO
4493,23/12/2024,66.74,truck31,GB,HVO
4494,23/12/2024,104.16,truck22,GB,HVO


In [7]:
# Diesel (from petrol stations)
diesel = pd.read_csv('GB UK fuels data(1).csv')

In [8]:
# Checking it's loaded correctly
diesel

,Date,Card Type,Product,Site No,Quantity,Unit Price,Cost,Unnamed: 7,Unnamed: 8
0,20/12/2024,BP BUNKER,BP CLEANER DIESEL,159104,54.97,1.190831,65.46,truck33,NaN
1,20/12/2024,BP BUNKER,BP AD BLUE,159104,27.19,1.465980,39.86,truck33,NaN
2,20/12/2024,UK FUELS,DIESEL,10118,61.11,1.091474,66.70,truck46,NaN
3,19/12/2024,UK FUELS,DIESEL,8945,100.01,1.091391,109.15,truck50,NaN
4,17/12/2024,BP BUNKER,BP AD BLUE,450511,26.91,1.264586,34.03,truck54,NaN
...,...,...,...,...,...,...,...,...,...
508,08/01/2024,UK FUELS,DIESEL,8944,150.00,1.286400,192.96,truck20,NaN
509,04/01/2024,UK FUELS,DIESEL,9386,55.88,1.309055,73.15,NaN,NaN
510,03/01/2024,BP BUNKER,BP CLEANER DIESEL,600035,266.54,1.302093,347.06,truck25,NaN
511,03/01/2024,BP BUNKER,BP AD BLUE,600035,21.11,1.447655,30.56,truck25,NaN


Cleaning the data:
- changing column `Unnamed: 7` to `REG` for consistency
- dropping all empty columns or with variables which are of no interest to the aim of this report
- dropping all rows where the vehicle is unknown
- dropping all rows with AdBlue (diesel exhaust liquid), as we are only interested in fuel


In [9]:
diesel = diesel.rename(columns={'Unnamed: 7': 'REG'})
diesel = diesel.drop(columns = ['Unnamed: 8', 'Card Type', 'Site No', 'Unit Price', 'Cost'])
diesel = diesel.dropna(axis = 0)
diesel = diesel[diesel['Product'] != 'BP AD BLUE']
diesel = diesel.drop(columns = ['Product'])
diesel

,Date,Quantity,REG
0,20/12/2024,54.97,truck33
2,20/12/2024,61.11,truck46
3,19/12/2024,100.01,truck50
5,17/12/2024,252.35,truck54
6,17/12/2024,237.64,truck50
...,...,...,...
506,09/01/2024,275.42,truck41
507,08/01/2024,56.25,truck39
508,08/01/2024,150.00,truck20
510,03/01/2024,266.54,truck25


In [55]:
# Miles travelled per vehicle
mileage = pd.read_excel('miles travelled 2024(3).xlsx')

In [56]:
# Checking it's loaded correctly
mileage

,truck,date,location,miles travelled,CLASS
0,truck1,02/01/24,GB,0.099432,HGV
1,truck2,02/01/24,GB,0.123864,HGV
2,truck2,02/01/24,GB,0.330114,HGV
3,truck2,02/01/24,GB,14.300000,HGV
4,truck2,02/01/24,GB,1.100000,HGV
...,...,...,...,...,...
51054,truck46,19/12/24,GB,0.287500,HGV
51055,truck46,19/12/24,GB,0.167045,HGV
51056,truck35,19/12/24,GB,0.231250,HGV
51057,truck35,19/12/24,GB,0.300000,HGV


In [14]:
# Fuel bought abroad
fuel_abroad = pd.read_excel('Fuel_Report_2025-05-23 2 cleared(1).xlsx')

In [15]:
# Checking it's loaded correctly
fuel_abroad

,Transaction time,Product group,Product,Product code,Sales,Unit,Price per unit,Total net,Total gross,Total gross (invoice currency),VAT,Discount net,Service fee net,country
0,2024-01-25 07:29:00,truck3,Diesel,WA0009,258.03,L,1.3975,360.588,432.71,370.54,72.122,0.000,0.000,GB
1,2024-07-16 14:33:40,truck19,Diesel,WA0009,62.09,L,1.8019,115.010,138.01,122.97,23.000,-0.620,3.750,FR
2,2024-07-02 07:04:00,truck16,Diesel,WA0009,286.58,L,1.6361,454.688,568.36,481.74,113.672,-30.548,16.353,DK
3,2024-04-24 07:26:00,truck47,Diesel,WA0009,402.19,L,1.6966,697.790,830.36,756.26,132.570,-7.240,22.660,DE
4,2024-12-13 14:39:00,truck35,Diesel,WA0009,300.47,L,1.3622,393.727,472.47,391.63,78.743,-15.587,0.000,GB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,2024-09-04 09:18:00,truck36,Diesel,WA0009,333.41,L,1.6377,557.620,669.13,597.82,111.510,-6.670,18.280,FR
1258,2024-03-25 15:57:09,truck10,Diesel,WA0009,309.32,L,1.5375,491.910,605.05,549.51,113.140,0.000,16.320,IE
1259,2024-06-18 04:04:52,truck36,Diesel Special,WA0028,198.14,L,1.7554,355.800,423.40,379.08,67.600,-3.570,11.550,DE
1260,2024-10-21 05:11:39,truck15,Diesel,WA0009,204.14,L,1.2797,265.608,332.01,276.43,66.402,-4.736,9.110,SE


Cleaning the data:
- dropping all rows where the vehicle is unknown
- dropping all columns with variables which are of no interest to the aim of this report

In [16]:
fuel_abroad = fuel_abroad.dropna(axis = 0)
cols = [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13]
fuel_abroad = fuel_abroad.drop(fuel_abroad.columns[cols], axis=1)
fuel_abroad

,Transaction time,Product group,Sales
0,2024-01-25 07:29:00,truck3,258.03
1,2024-07-16 14:33:40,truck19,62.09
2,2024-07-02 07:04:00,truck16,286.58
3,2024-04-24 07:26:00,truck47,402.19
4,2024-12-13 14:39:00,truck35,300.47
...,...,...,...
1257,2024-09-04 09:18:00,truck36,333.41
1258,2024-03-25 15:57:09,truck10,309.32
1259,2024-06-18 04:04:52,truck36,198.14
1260,2024-10-21 05:11:39,truck15,204.14


## Grouping by week ##

In [18]:
# Converting the dates
HVO_tank['Date_converted'] = pd.to_datetime(HVO_tank['DATE'], dayfirst=True, errors='coerce')
HVO_tank.loc[HVO_tank.index, 'Week'] = HVO_tank['Date_converted'].dt.isocalendar().week

In [19]:
HVO_tank

,DATE,LITRES,REG,Country,Type,Date_converted,Week
0,02/01/2024,67.31,truck32,GB,HVO,2024-01-02,1
1,02/01/2024,45.97,truck29,GB,HVO,2024-01-02,1
2,02/01/2024,52.02,truck24,GB,HVO,2024-01-02,1
3,02/01/2024,236.74,truck6,GB,HVO,2024-01-02,1
4,02/01/2024,329.21,truck4,GB,HVO,2024-01-02,1
...,...,...,...,...,...,...,...
4491,23/12/2024,69.20,truck30,GB,HVO,2024-12-23,52
4492,23/12/2024,63.23,truck32,GB,HVO,2024-12-23,52
4493,23/12/2024,66.74,truck31,GB,HVO,2024-12-23,52
4494,23/12/2024,104.16,truck22,GB,HVO,2024-12-23,52


In [20]:
# Dropping the last row as the fuel would be used in 2025
HVO_tank.drop([4498])

,DATE,LITRES,REG,Country,Type,Date_converted,Week
0,02/01/2024,67.31,truck32,GB,HVO,2024-01-02,1
1,02/01/2024,45.97,truck29,GB,HVO,2024-01-02,1
2,02/01/2024,52.02,truck24,GB,HVO,2024-01-02,1
3,02/01/2024,236.74,truck6,GB,HVO,2024-01-02,1
4,02/01/2024,329.21,truck4,GB,HVO,2024-01-02,1
...,...,...,...,...,...,...,...
4490,23/12/2024,137.10,truck22,GB,HVO,2024-12-23,52
4491,23/12/2024,69.20,truck30,GB,HVO,2024-12-23,52
4492,23/12/2024,63.23,truck32,GB,HVO,2024-12-23,52
4493,23/12/2024,66.74,truck31,GB,HVO,2024-12-23,52


In [21]:
HVO_tank = HVO_tank.drop(columns = ['DATE'])

In [22]:
# Converting the dates
diesel['Date_converted'] = pd.to_datetime(diesel['Date'], format="%d/%m/%Y", errors='coerce')
diesel.loc[diesel.index, 'Week'] = diesel['Date_converted'].dt.isocalendar().week

In [23]:
diesel

,Date,Quantity,REG,Date_converted,Week
0,20/12/2024,54.97,truck33,2024-12-20,51
2,20/12/2024,61.11,truck46,2024-12-20,51
3,19/12/2024,100.01,truck50,2024-12-19,51
5,17/12/2024,252.35,truck54,2024-12-17,51
6,17/12/2024,237.64,truck50,2024-12-17,51
...,...,...,...,...,...
506,09/01/2024,275.42,truck41,2024-01-09,2
507,08/01/2024,56.25,truck39,2024-01-08,2
508,08/01/2024,150.00,truck20,2024-01-08,2
510,03/01/2024,266.54,truck25,2024-01-03,1


In [24]:
diesel = diesel.drop(columns = ['Date', 'Date_converted'])

In [57]:
# Converting the dates
mileage['Date_converted'] = pd.to_datetime(mileage['date'], format="%d/%m/%y", errors='coerce')
mileage.loc[mileage.index, 'Week'] = mileage['Date_converted'].dt.isocalendar().week

In [58]:
mileage

,truck,date,location,miles travelled,CLASS,Date_converted,Week
0,truck1,02/01/24,GB,0.099432,HGV,2024-01-02,1
1,truck2,02/01/24,GB,0.123864,HGV,2024-01-02,1
2,truck2,02/01/24,GB,0.330114,HGV,2024-01-02,1
3,truck2,02/01/24,GB,14.300000,HGV,2024-01-02,1
4,truck2,02/01/24,GB,1.100000,HGV,2024-01-02,1
...,...,...,...,...,...,...,...
51054,truck46,19/12/24,GB,0.287500,HGV,2024-12-19,51
51055,truck46,19/12/24,GB,0.167045,HGV,2024-12-19,51
51056,truck35,19/12/24,GB,0.231250,HGV,2024-12-19,51
51057,truck35,19/12/24,GB,0.300000,HGV,2024-12-19,51


In [59]:
mileage = mileage.drop(columns = ['date', 'Date_converted'])

In [28]:
# Converting the dates
fuel_abroad['Date_converted'] = pd.to_datetime(fuel_abroad['Transaction time'], errors='coerce')
fuel_abroad.loc[fuel_abroad.index, 'Week'] = fuel_abroad['Date_converted'].dt.isocalendar().week

In [29]:
fuel_abroad

,Transaction time,Product group,Sales,Date_converted,Week
0,2024-01-25 07:29:00,truck3,258.03,2024-01-25 07:29:00,4
1,2024-07-16 14:33:40,truck19,62.09,2024-07-16 14:33:40,29
2,2024-07-02 07:04:00,truck16,286.58,2024-07-02 07:04:00,27
3,2024-04-24 07:26:00,truck47,402.19,2024-04-24 07:26:00,17
4,2024-12-13 14:39:00,truck35,300.47,2024-12-13 14:39:00,50
...,...,...,...,...,...
1257,2024-09-04 09:18:00,truck36,333.41,2024-09-04 09:18:00,36
1258,2024-03-25 15:57:09,truck10,309.32,2024-03-25 15:57:09,13
1259,2024-06-18 04:04:52,truck36,198.14,2024-06-18 04:04:52,25
1260,2024-10-21 05:11:39,truck15,204.14,2024-10-21 05:11:39,43


In [30]:
fuel_abroad = fuel_abroad.drop(columns = ['Transaction time', 'Date_converted'])

## Calculating fuel per vehicle per week ##

In [31]:
HVO_tank = HVO_tank.groupby(['REG', 'Week'])['LITRES'].sum().reset_index()

In [32]:
HVO_tank

,REG,Week,LITRES
0,truck1,2,720.70
1,truck1,3,587.42
2,truck1,4,635.69
3,truck1,5,613.99
4,truck1,6,455.06
...,...,...,...
1668,truck9,45,546.36
1669,truck9,46,573.50
1670,truck9,47,990.79
1671,truck9,48,409.72


In [33]:
diesel = diesel.groupby(['REG', 'Week'])['Quantity'].sum().reset_index()

In [34]:
diesel

,REG,Week,Quantity
0,truck10,10,50.00
1,truck10,23,50.01
2,truck10,25,100.00
3,truck13,19,100.02
4,truck13,26,250.02
...,...,...,...
344,truck8,26,333.82
345,truck8,39,69.19
346,truck8,44,244.96
347,truck8,46,150.08


In [35]:
fuel_usage = pd.merge(HVO_tank, diesel, on=['REG', 'Week'], how='outer')

Once the two dataframes have been merged, we will need to replace missing values with zeroes (as a missing value means this type of fuel was not used by the vehicle in that week). Also renaming the columns for clarity:

In [36]:
fuel_usage = fuel_usage.fillna(0)
fuel_usage = fuel_usage.rename(columns={'LITRES': 'HVO', 'Quantity': 'Diesel'})
fuel_usage

,REG,Week,HVO,Diesel
0,truck1,2,720.70,0.0
1,truck1,3,587.42,0.0
2,truck1,4,635.69,0.0
3,truck1,5,613.99,0.0
4,truck1,6,455.06,0.0
...,...,...,...,...
1738,truck9,45,546.36,0.0
1739,truck9,46,573.50,0.0
1740,truck9,47,990.79,0.0
1741,truck9,48,409.72,0.0


These data can now be merged with the fuel bought abroad:

In [39]:
fuel_abroad = fuel_abroad.rename(columns={'Product group': 'REG'})

In [48]:
fuel_all = pd.merge(fuel_usage, fuel_abroad, on=['REG', 'Week'], how='outer')
fuel_all = fuel_all.fillna(0)

# Adding diesel bought in Britain and abroad
fuel_all['Diesel'] = fuel_all['Diesel'] + fuel_all['Sales']
fuel_all = fuel_all.drop(columns = 'Sales')

# Grouping by vehicles and weeks again
fuel_all = fuel_all.groupby(['REG', 'Week'])[['HVO', 'Diesel']].sum().reset_index()

fuel_all

,REG,Week,HVO,Diesel
0,truck1,2,720.70,0.00
1,truck1,3,587.42,0.00
2,truck1,4,635.69,0.00
3,truck1,5,613.99,0.00
4,truck1,6,455.06,0.00
...,...,...,...,...
2109,truck9,45,546.36,0.00
2110,truck9,46,573.50,0.00
2111,truck9,47,990.79,0.00
2112,truck9,48,409.72,285.66


## Calculating mileage per vehicle per week ##

All that needs to be done is to join the mileage data with fuel consumption. Looking at the dataframe again:

In [60]:
mileage

,truck,location,miles travelled,CLASS,Week
0,truck1,GB,0.099432,HGV,1
1,truck2,GB,0.123864,HGV,1
2,truck2,GB,0.330114,HGV,1
3,truck2,GB,14.300000,HGV,1
4,truck2,GB,1.100000,HGV,1
...,...,...,...,...,...
51054,truck46,GB,0.287500,HGV,51
51055,truck46,GB,0.167045,HGV,51
51056,truck35,GB,0.231250,HGV,51
51057,truck35,GB,0.300000,HGV,51


In [61]:
# Grouping by vehicles, weeks, class and countries to calculate the total mileages in Britain and abroad
mileage = mileage.groupby(['truck', 'location', 'CLASS', 'Week'])['miles travelled'].sum().reset_index()
mileage

,truck,location,CLASS,Week,miles travelled
0,truck1,GB,HGV,1,363.493182
1,truck1,GB,HGV,2,1485.336932
2,truck1,GB,HGV,3,1045.735795
3,truck1,GB,HGV,4,1442.570455
4,truck1,GB,HGV,5,1070.335227
...,...,...,...,...,...
4261,truck9,GB,HGV,48,1070.407955
4262,truck9,GB,HGV,49,582.783523
4263,truck9,GB,HGV,50,0.426705
4264,truck9,IE,HGV,5,544.014773


To process the data, we are interested in mileages in each country. First, we need a list of all countries in the `location` column:

In [62]:
cols = mileage['location'].unique()
cols

array(['GB', '—', '41', '44', 'IE', 'le', 'AT', 'BE', 'DE', 'DK', 'ES',
       'FR', 'IT', 'NL', 'NO', 'PT', 'SE', 'CH', 'LU', '57', 'PL', '29',
       '59', 'HR', 'SI'], dtype=object)

The non-country values, such as 'le' and '41' are related to data input errors. The data providers informed me all these values are related to UK locations, hence they will be replaced with 'GB'.

In [63]:
mileage = mileage.replace({'location': {'—': 'GB', '41': 'GB', '44': 'GB', 'le': 'GB', '57': 'GB', '29': 'GB', '59': 'GB'}})
cols = mileage['location'].unique()

In [64]:
from numpy import where

# Adding columns per each country on the list
for col in cols:
    mileage[col] = where(mileage['location'] == col, mileage['miles travelled'], 0)

# Dropping the obsolete columns
#mileage = mileage.drop(columns = ['location', 'CLASS'], 'miles travelled')

# Checking if the dataframe looks correctly
mileage

,truck,location,CLASS,Week,miles travelled,GB,IE,AT,BE,DE,...,IT,NL,NO,PT,SE,CH,LU,PL,HR,SI
0,truck1,GB,HGV,1,363.493182,363.493182,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,truck1,GB,HGV,2,1485.336932,1485.336932,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,truck1,GB,HGV,3,1045.735795,1045.735795,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,truck1,GB,HGV,4,1442.570455,1442.570455,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,truck1,GB,HGV,5,1070.335227,1070.335227,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,truck9,GB,HGV,48,1070.407955,1070.407955,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4262,truck9,GB,HGV,49,582.783523,582.783523,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4263,truck9,GB,HGV,50,0.426705,0.426705,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4264,truck9,IE,HGV,5,544.014773,0.000000,544.014773,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
# Changing the column names for consistency
mileage = mileage.rename(columns={'truck': 'REG'})
mileage = mileage.rename(columns={'miles travelled': 'miles'})

In [66]:
# Removing the (now obsolete) location column
mileage = mileage.drop(columns = 'location')

In [67]:
# Grouping by vehicles, weeks and class
mileage = mileage.groupby(['REG', 'CLASS', 'Week'])[[
    'miles', 'GB', 'IE', 'AT', 'BE', 'DE', 'DK', 'ES', 'FR', 'IT', 'NL', 'NO', 'PT', 'SE', 'CH', 'LU', 'PL', 'HR', 'SI']].sum().reset_index()
mileage

,REG,CLASS,Week,miles,GB,IE,AT,BE,DE,DK,...,IT,NL,NO,PT,SE,CH,LU,PL,HR,SI
0,truck1,HGV,1,363.493182,363.493182,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,truck1,HGV,2,1485.336932,1485.336932,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,truck1,HGV,3,1045.735795,1045.735795,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,truck1,HGV,4,1442.570455,1442.570455,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,truck1,HGV,5,1070.335227,1070.335227,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463,truck9,HGV,46,1437.104545,1437.104545,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2464,truck9,HGV,47,2187.004545,2187.004545,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2465,truck9,HGV,48,1070.407955,1070.407955,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466,truck9,HGV,49,582.783523,582.783523,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Joining the data together ##

In [68]:
fuel_data = pd.merge(fuel_all, mileage, on=['REG', 'Week'], how='outer')

In [69]:
fuel_data

,REG,Week,HVO,Diesel,CLASS,miles,GB,IE,AT,BE,...,IT,NL,NO,PT,SE,CH,LU,PL,HR,SI
0,truck1,1,NaN,NaN,HGV,363.493182,363.493182,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,truck1,2,720.70,0.00,HGV,1485.336932,1485.336932,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,truck1,3,587.42,0.00,HGV,1045.735795,1045.735795,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,truck1,4,635.69,0.00,HGV,1442.570455,1442.570455,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,truck1,5,613.99,0.00,HGV,1070.335227,1070.335227,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2521,truck9,46,573.50,0.00,HGV,1437.104545,1437.104545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2522,truck9,47,990.79,0.00,HGV,2187.004545,2187.004545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2523,truck9,48,409.72,285.66,HGV,1070.407955,1070.407955,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2524,truck9,49,240.00,0.00,HGV,582.783523,582.783523,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# Grouping by vehicles, weeks and class
fuel_data = fuel_data.groupby(['REG', 'Week'])[[
    'HVO', 'Diesel', 'CLASS', 'miles', 'GB', 'IE', 'AT', 'BE', 'DE', 'DK', 'ES', 'FR', 'IT', 'NL', 'NO', 'PT', 'SE', 'CH', 'LU', 'PL', 'HR', 'SI']].sum().reset_index()
fuel_data

,REG,Week,HVO,Diesel,CLASS,miles,GB,IE,AT,BE,...,IT,NL,NO,PT,SE,CH,LU,PL,HR,SI
0,truck1,1,0.00,0.00,HGV,363.493182,363.493182,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,truck1,2,720.70,0.00,HGV,1485.336932,1485.336932,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,truck1,3,587.42,0.00,HGV,1045.735795,1045.735795,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,truck1,4,635.69,0.00,HGV,1442.570455,1442.570455,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,truck1,5,613.99,0.00,HGV,1070.335227,1070.335227,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2521,truck9,46,573.50,0.00,HGV,1437.104545,1437.104545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2522,truck9,47,990.79,0.00,HGV,2187.004545,2187.004545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2523,truck9,48,409.72,285.66,HGV,1070.407955,1070.407955,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2524,truck9,49,240.00,0.00,HGV,582.783523,582.783523,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
# Checking for any missing values left
fuel_data.isna().sum()

REG       0
Week      0
HVO       0
Diesel    0
CLASS     0
miles     0
GB        0
IE        0
AT        0
BE        0
DE        0
DK        0
ES        0
FR        0
IT        0
NL        0
NO        0
PT        0
SE        0
CH        0
LU        0
PL        0
HR        0
SI        0
dtype: int64

As we are investigating fuel efficiency, we are adding a calculation of miles per gallon, based on the fuel used and total mileage each week:

In [74]:
fuel_data['fuel'] = fuel_data['HVO'] + fuel_data['Diesel']

If the fuel used is showing as 0, it is an error, so the rows need to be removed.

In [77]:
fuel_data = fuel_data[fuel_data['fuel'] != 0].copy()

In [78]:
# one gallon = 4.54609 litres
fuel_data['mpg'] = fuel_data['miles'] / fuel_data['fuel'] * 4.54609

In [79]:
# Checking if everything looks correct before we export it
fuel_data

,REG,Week,HVO,Diesel,CLASS,miles,GB,IE,AT,BE,...,NO,PT,SE,CH,LU,PL,HR,SI,fuel,mpg
1,truck1,2,720.70,0.00,HGV,1485.336932,1485.336932,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720.70,9.369329
2,truck1,3,587.42,0.00,HGV,1045.735795,1045.735795,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,587.42,8.093032
3,truck1,4,635.69,0.00,HGV,1442.570455,1442.570455,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,635.69,10.316436
4,truck1,5,613.99,0.00,HGV,1070.335227,1070.335227,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,613.99,7.924950
5,truck1,6,455.06,0.00,HGV,1209.755682,1209.755682,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455.06,12.085567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2520,truck9,45,546.36,0.00,HGV,1274.102273,1274.102273,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,546.36,10.601405
2521,truck9,46,573.50,0.00,HGV,1437.104545,1437.104545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,573.50,11.391816
2522,truck9,47,990.79,0.00,HGV,2187.004545,2187.004545,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,990.79,10.034739
2523,truck9,48,409.72,285.66,HGV,1070.407955,1070.407955,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,695.38,6.997859


## Exporting the data for statistical analysis in R ##

In [80]:
fuel_data.to_csv('fuel.csv')